In [1]:
import time
import numpy as np
import torch
import torchaudio

import matplotlib.pyplot as plt
import seaborn as sns

from itertools import groupby
from operator import itemgetter
from matplotlib.collections import LineCollection

device = 'cuda'

In [2]:
from speech_parser import Audio
from speech_parser import SileroVAD
from speech_parser import OnlineSpeakerClustering
from speech_parser import MSDD

/home/raid/miniconda3/envs/nemo/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [15]:
x = torch.tensor([[0.1, 0.2, 0.3, -0.4, 0.5], [0.1, 0.2, 0.3, -0.4, 0.5], [0.1, 0.2, 0.3, 0, 0.5], [0.1, 0.2, 0.3, -0.4, 0.5], [0.1, 0.2, 0.3, -0.4, 0.5]])
y = torch.tensor([1, 2, 3, 2, 3])

In [17]:
unique_labels = y.unique()

torch.stack([x[y == label].mean() for label in unique_labels])

tensor([0.1400, 0.1400, 0.1800])

In [3]:
from utils import load_audio

In [4]:
msdd = MSDD(
    threshold=0.8
)
titanet_l = msdd.speech_embedding_model
vad = SileroVAD(
    threshold=0.5
)
osc = OnlineSpeakerClustering()

[NeMo I 2025-02-26 17:19:13 cloud:58] Found existing object /home/raid/.cache/torch/NeMo/NeMo_2.1.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2025-02-26 17:19:13 cloud:64] Re-using file from: /home/raid/.cache/torch/NeMo/NeMo_2.1.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo
[NeMo I 2025-02-26 17:19:13 common:826] Instantiating model from pre-trained checkpoint


[NeMo W 2025-02-26 17:19:15 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: true
    
[NeMo W 2025-02-26 17:19:15 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: false
    
[NeMo W 2025-02-26 17:19:15 modelPT:189] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2025-02-26 17:19:15 features:305] PADDING: 16
[NeMo I 2025-02-26 17:19:15 features:305] PADDING: 16
[NeMo I 2025-02-26 17:19:16 save_restore_connector:275] Model EncDecDiarLabelModel was successfully restored from /home/raid/.cache/torch/NeMo/NeMo_2.1.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.


In [5]:
# waveform, sr = load_audio('test.wav')
waveform, sr = load_audio('toefl_eg.mp3')
waveform = waveform[0]

In [6]:
scales = [1.5, 1.25, 1.0, 0.75, 0.5]
hops = [scale/4 for scale in scales]
a = Audio(
    scales, 
    hops, 
    speech_embedding_model=titanet_l,
    voice_activity_detection_model=vad,
    multi_scale_diarization_model=msdd,
    speaker_clustering=osc
)

In [7]:
proba, labels = a(waveform[:500_000])

In [8]:
proba, labels = a(waveform[500_000:1_000_000])

In [9]:
proba, labels = a(waveform[1_000_000:1_500_000])

In [10]:
proba

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

In [11]:
proba, labels = a(waveform[1_500_000:2_000_000])

In [12]:
proba, labels = a(waveform[2_000_000:2_500_000])

In [13]:
proba, labels = a(waveform[2_500_000:3_000_000])

In [14]:
proba

tensor([[0.1356, 0.1184, 0.1050, 0.0728, 0.0698, 0.1083, 0.1571, 0.1815, 0.1480,
         0.1260, 0.1000, 0.0785, 0.0692, 0.0671, 0.0763, 0.1061, 0.1279, 0.1478,
         0.1541, 0.1397, 0.1148, 0.0859, 0.0604, 0.0574, 0.0763, 0.1102, 0.1314,
         0.1270, 0.1104, 0.0994, 0.0988, 0.1265, 0.1472, 0.1454, 0.1201, 0.0955,
         0.0785, 0.0794, 0.0824, 0.0801, 0.0832, 0.0962, 0.1112, 0.1123, 0.1176,
         0.1222, 0.1194, 0.1233, 0.1218, 0.1262, 0.1778, 0.2016, 0.2492, 0.3240,
         0.3829, 0.3947, 0.3751, 0.4090, 0.4461, 0.4720, 0.4768, 0.4799, 0.4867,
         0.4858, 0.4772, 0.4742, 0.4718, 0.4533, 0.4555, 0.4554, 0.4484, 0.4550,
         0.4608, 0.4697, 0.4680, 0.4652, 0.4675, 0.4713, 0.4619, 0.4647, 0.4651,
         0.4572, 0.4566, 0.4589, 0.4768, 0.4685, 0.4871, 0.4799, 0.4869, 0.4577,
         0.4359, 0.3838, 0.3241, 0.3079, 0.2992, 0.3055, 0.3413, 0.3730, 0.3566,
         0.1730, 0.1189, 0.1059, 0.0889, 0.0762, 0.0757, 0.0776, 0.0815, 0.0763,
         0.0684, 0.0709, 0.0

In [33]:
t

(tensor([[1.0000, 1.0000, 1.0000,  ..., 0.1265, 0.1363, 0.1547],
         [0.4071, 0.4726, 0.5145,  ..., 1.0000, 1.0000, 1.0000],
         [0.4454, 0.3957, 0.3099,  ..., 0.5040, 0.5040, 0.5122]]),
 tensor([[1., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [10]:
proba, labels = a(waveform[1_000_000:4_000_000])

In [11]:
a(waveform[4_000_000:10_000_000])

(tensor([[0.5779, 0.6387, 0.7030,  ..., 0.7686, 0.7629, 0.7463],
         [1.0000, 1.0000, 1.0000,  ..., 0.7863, 0.7777, 0.7710],
         [0.7010, 0.7149, 0.7094,  ..., 1.0000, 1.0000, 1.0000],
         [0.3367, 0.3425, 0.3632,  ..., 0.5455, 0.5568, 0.5384],
         [0.3893, 0.3460, 0.3051,  ..., 0.3876, 0.3940, 0.3771],
         [0.6612, 0.6315, 0.5984,  ..., 0.1415, 0.1071, 0.0864]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [17]:
merged = a.get_merged_speaker_segments()

In [18]:
merged

[SpeakerSegment(start=0, end=17.625, data=tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.0015, -0.0009, -0.0007]), mask=tensor([0., 0., 0.,  ..., 0., 0., 0.]), speaker=0, transcription=None),
 SpeakerSegment(start=17.875, end=50.375, data=tensor([-0.0037, -0.0012,  0.0005,  ...,  0.0037,  0.0032,  0.0023]), mask=tensor([0., 0., 0.,  ..., 0., 0., 0.]), speaker=0, transcription=None),
 SpeakerSegment(start=50.875, end=61.875, data=tensor([ 1.6737e-03,  3.6514e-04, -5.9805e-05,  ..., -1.1843e-03,
         -1.0885e-03, -7.7725e-04]), mask=tensor([0., 0., 0.,  ..., 0., 0., 0.]), speaker=0, transcription=None),
 SpeakerSegment(start=62.375, end=93.0, data=tensor([-0.0970, -0.0802, -0.1046,  ..., -0.0864, -0.0612, -0.0339]), mask=tensor([1., 1., 1.,  ..., 1., 1., 1.]), speaker=0, transcription=None),
 SpeakerSegment(start=93.625, end=124.375, data=tensor([ 0.0049,  0.0063,  0.0076,  ..., -0.0357, -0.0246, -0.0219]), mask=tensor([1., 1., 1.,  ..., 1., 1., 1.]), speaker=0, transcription=None),
 Sp

In [32]:
i = 50
torchaudio.save('segmented.wav', merged[i].data.unsqueeze(0).cpu(), 16_000)
merged[i].speaker

2

In [33]:
merged[i].end

542.5

In [18]:
timeline = [segment.speakers for segment in a.base_scale_segments]

In [19]:
def process_timeline(data):
    # Convert None to empty list for consistency
    timeline = [[] if x is None else sorted(x) for x in data]
    return timeline

def merge_segments(timeline):
    merged_segments = []
    
    # Find continuous segments for each speaker
    for speaker in set([spk for t in timeline for spk in t]):
        start_idx = None
        
        for t, speakers in enumerate(timeline):
            if speaker in speakers:
                if start_idx is None:
                    start_idx = t
            elif start_idx is not None:
                # Add segment
                merged_segments.append({
                    'speaker': speaker,
                    'start': start_idx * 0.25,  # Convert to seconds
                    'end': t * 0.25,            # Convert to seconds
                    'duration': (t - start_idx) * 0.25
                })
                start_idx = None
        
        # Handle segment that goes until the end
        if start_idx is not None:
            merged_segments.append({
                'speaker': speaker,
                'start': start_idx * 0.25,
                'end': len(timeline) * 0.25,
                'duration': (len(timeline) - start_idx) * 0.25
            })
    
    return sorted(merged_segments, key=lambda x: (x['start'], x['speaker']))

def generate_rttm(segments):
    rttm_lines = []
    
    for seg in segments:
        rttm_line = f"SPEAKER unknown 1 {seg['start']:.3f} {seg['duration']:.3f} <NA> <NA> SPEAKER_{seg['speaker']} <NA> <NA>"
        rttm_lines.append(rttm_line)
    
    return "\n".join(rttm_lines)

def create_visualization(segments, total_duration):
    plt.figure(figsize=(20, 5))
    
    # Get unique speakers and assign them y-coordinates
    unique_speakers = sorted(set(seg['speaker'] for seg in segments))
    speaker_to_y = {speaker: i for i, speaker in enumerate(unique_speakers)}
    
    # Create line segments for each speaker
    for speaker in unique_speakers:
        speaker_segments = [seg for seg in segments if seg['speaker'] == speaker]
        
        for seg in speaker_segments:
            plt.hlines(
                y=speaker_to_y[seg['speaker']],
                xmin=seg['start'],
                xmax=seg['end'],
                linewidth=4,
                label=f"Speaker {speaker}"
            )
    
    # Set axis limits to show the full timeline
    plt.xlim(0, total_duration)
    plt.ylim(-0.5, len(unique_speakers) - 0.5)
    
    # Set y-axis labels
    plt.yticks(
        range(len(unique_speakers)),
        [f"Speaker {speaker}" for speaker in unique_speakers]
    )
    
    # Remove duplicate labels in legend
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Customize the plot
    plt.xlabel("Time (seconds)")
    plt.ylabel("Speakers")
    plt.title("Speaker Diarization Timeline")
    plt.grid(True, alpha=0.3)
    
    # Add some padding to the plot
    plt.margins(x=0.02)
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    # Save the plot
    plt.savefig('speaker_diarization.png', bbox_inches='tight')
    plt.close()

# Use your existing code to process the data and generate segments
timeline = process_timeline(timeline)
merged_segments = merge_segments(timeline)

# Calculate total duration in seconds
total_duration = len(timeline) * 0.25

# Generate RTTM
rttm_content = generate_rttm(merged_segments)
with open('output.rttm', 'w') as f:
    f.write(rttm_content)

# Create visualization with total duration
create_visualization(merged_segments, total_duration)

# Print segments for verification
for seg in merged_segments:
    print(f"Speaker {seg['speaker']}: {seg['start']:.2f}s - {seg['end']:.2f}s (duration: {seg['duration']:.2f}s)")

Speaker 0: 62.00s - 120.00s (duration: 58.00s)
Speaker 0: 120.25s - 134.00s (duration: 13.75s)
Speaker 0: 135.75s - 137.75s (duration: 2.00s)
Speaker 1: 135.75s - 140.25s (duration: 4.50s)
Speaker 1: 140.75s - 145.00s (duration: 4.25s)
Speaker 2: 145.50s - 155.25s (duration: 9.75s)
Speaker 2: 155.50s - 156.50s (duration: 1.00s)
Speaker 1: 156.75s - 160.00s (duration: 3.25s)
Speaker 2: 159.00s - 168.75s (duration: 9.75s)
Speaker 1: 168.50s - 178.25s (duration: 9.75s)
Speaker 2: 171.75s - 172.75s (duration: 1.00s)
Speaker 2: 175.25s - 175.75s (duration: 0.50s)
Speaker 2: 178.00s - 181.75s (duration: 3.75s)
Speaker 2: 182.00s - 187.25s (duration: 5.25s)
Speaker 1: 187.50s - 191.25s (duration: 3.75s)
Speaker 2: 191.00s - 191.25s (duration: 0.25s)
Speaker 1: 191.50s - 191.75s (duration: 0.25s)
Speaker 2: 191.50s - 197.00s (duration: 5.50s)
Speaker 1: 196.50s - 206.75s (duration: 10.25s)
Speaker 1: 207.00s - 215.75s (duration: 8.75s)
Speaker 2: 215.25s - 220.25s (duration: 5.00s)
Speaker 1: 